Some physical quantities are directly related to other unitful quantities by a constant, but otherwise do not have the same units. To facilitate conversions between these quantities, `yt` implements a system of unit equivalencies (inspired by the [AstroPy implementation](http://docs.astropy.org/en/latest/units/equivalencies.html)). The possible unit equivalencies are:

* `"thermal"`: conversions between temperature and energy ($E = k_BT$)
* `"spectral"`: conversions between wavelength, frequency, and energy for photons ($E = h\nu = hc/\lambda, c = \lambda\nu$)
* `"mass_energy"`: conversions between mass and energy ($E = mc^2$)
* `"lorentz"`: conversions between velocity and Lorentz factor ($\gamma = 1/\sqrt{1-(v/c)^2}$)
* `"schwarzschild"`: conversions between mass and Schwarzschild radius ($R_S = 2GM/c^2$)
* `"compton"`: conversions between mass and Compton wavelength ($\lambda = h/mc$)

The following unit equivalencies only apply under conditions applicable for an ideal gas with a constant mean molecular weight $\mu$ and ratio of specific heats $\gamma$:

* `"number_density"`: conversions between density and number density ($n = \rho/\mu{m_p}$)
* `"sound_speed"`: conversions between temperature and sound speed for an ideal gas ($c_s^2 = \gamma{k_BT}/\mu{m_p}$)

A `YTArray` or `YTQuantity` can be converted to an equivalent using `in_units` (previously described in [Fields and Unit Conversion](fields_and_unit_conversion.html)), where the unit and the equivalence name are provided as additional arguments:

In [ ]:
import yt
from yt import YTQuantity
import numpy as np

ds = yt.load('IsolatedGalaxy/galaxy0030/galaxy0030')

dd = ds.all_data()

print (dd["temperature"].in_units("erg", equivalence="thermal"))
print (dd["temperature"].in_units("eV", equivalence="thermal"))

# Rest energy of the proton
from yt.units import mp
E_p = mp.in_units("GeV", equivalence="mass_energy")
print (E_p)

Most equivalencies can go in both directions, without any information required other than the unit you want to convert to (this is not the case for the electromagnetic equivalencies, which we'll discuss later):

In [ ]:
from yt.units import clight
v = 0.1*clight
g = v.in_units("dimensionless", equivalence="lorentz")
print (g)
print (g.in_units("c", equivalence="lorentz"))

The previously described `to_value` method, which works like `in_units` except that it returns a bare NumPy array or floating-point number, also accepts equivalencies:

In [ ]:
print (dd["temperature"].to_value("erg", equivalence="thermal"))
print (mp.to_value("GeV", equivalence="mass_energy"))

### Special Equivalencies

Some equivalencies can take supplemental information. The `"number_density"` equivalence can take a custom mean molecular weight (default is $\mu = 0.6$):

In [ ]:
print (dd["density"].max())
print (dd["density"].in_units("cm**-3", equivalence="number_density").max())
print (dd["density"].in_units("cm**-3", equivalence="number_density", mu=0.75).max())

The `"sound_speed"` equivalence optionally takes the ratio of specific heats $\gamma$ and the mean molecular weight $\mu$ (defaults are $\gamma$ = 5/3, $\mu = 0.6$):

In [ ]:
print (dd["temperature"].in_units("km/s", equivalence="sound_speed").mean())
print (dd["temperature"].in_units("km/s", equivalence="sound_speed", gamma=4./3., mu=0.5).mean())

These options must be used with caution, and only if you know the underlying data adheres to these assumptions!

### Electromagnetic Equivalencies

Special, one-way equivalencies exist for converting between electromagnetic units in the cgs and SI unit systems. These exist since in the cgs system, electromagnetic units are comprised of the base units of seconds, grams and centimeters, whereas in the SI system Ampere is a base unit. For example, the dimensions of charge are completely different in the two systems:

In [ ]:
Q1 = YTQuantity(1.0,"C")
Q2 = YTQuantity(1.0,"esu")
print ("Q1 dims =", Q1.units.dimensions)
print ("Q2 dims =", Q2.units.dimensions)
print ("Q1 base units =", Q1.in_mks())
print ("Q2 base units =", Q2.in_cgs())

To convert from a cgs unit to an SI unit, use the "SI" equivalency:

In [ ]:
from yt.units import qp # the elementary charge in esu
qp_SI = qp.in_units("C", equivalence="SI") # convert to Coulombs
print (qp)
print (qp_SI)

To convert from an SI unit to a cgs unit, use the "CGS" equivalency:

In [ ]:
B = YTQuantity(1.0,"T") # magnetic field in Tesla
print (B, B.in_units("gauss", equivalence="CGS")) # convert to Gauss

Equivalencies exist between the SI and cgs dimensions of charge, current, magnetic field, electric potential, and resistance. As a neat example, we can convert current in Amperes and resistance in Ohms to their cgs equivalents, and then use them to calculate the "Joule heating" of a conductor with resistance $R$ and current $I$:

In [ ]:
I = YTQuantity(1.0,"A")
I_cgs = I.in_units("statA", equivalence="CGS")
R = YTQuantity(1.0,"ohm")
R_cgs = R.in_units("statohm", equivalence="CGS")
P = I**2*R
P_cgs = I_cgs**2*R_cgs

The dimensions of current and resistance in the two systems are completely different, but the formula gives us the power dissipated dimensions of energy per time, so the dimensions and the result should be the same, which we can check:

In [ ]:
print (P_cgs.units.dimensions == P.units.dimensions)
print (P.in_units("W"), P_cgs.in_units("W"))

### Determining Valid Equivalencies

If a certain equivalence does not exist for a particular unit, then an error will be thrown:

In [ ]:
from yt.utilities.exceptions import YTInvalidUnitEquivalence

try:
    x = v.in_units("angstrom", equivalence="spectral")
except YTInvalidUnitEquivalence as e:
    print (e)

You can check if a `YTArray` has a given equivalence with `has_equivalent`:

In [ ]:
print (mp.has_equivalent("compton"))
print (mp.has_equivalent("thermal"))

To list the equivalencies available for a given `YTArray` or `YTQuantity`, use the `list_equivalencies` method:

In [ ]:
E_p.list_equivalencies()